In [24]:
from binascii import hexlify, unhexlify


In [25]:
# 1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736
# ... has been XOR'd against a single character. Find the key, decrypt the message.

# You can do this by hand. But don't: write code to do it for you.

# How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.


In [26]:
def bxor(a, b):
    "bitwise XOR of bytestrings"
    return bytes([ x^y for (x,y) in zip(a, b)])


In [27]:
msg = b"yo wassup"

# random number btn 0-255 lets say 13
key = b"\x13"

# the byte string we will XOR the message against is usually called the "keystream"
keystream = key*len(msg)

bxor(msg,keystream)

b'j|3dr``fc'

In [28]:
ciphertext = unhexlify("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736")
ciphertext

b'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'

In [29]:
candidate_key = bytes([1])
keystream = candidate_key*len(ciphertext)

bxor(ciphertext, keystream)

b'\x1a66207>y\x14\x1a~*y502<y8y)6,7=y6?y;8:67'

In [30]:
ord('a'), ord('b'), ord('z')

(97, 98, 122)

In [31]:

ascii_text_chars = list(range(97, 122)) + [32]


In [32]:
[ x in ascii_text_chars for x in b"yo bro? "]

[True, True, True, True, True, True, False, True]

In [33]:
sum([ x in ascii_text_chars for x in b"yo bro?"])

6

In [34]:
def letter_ratio(input_bytes):
    nb_letters = sum([ x in ascii_text_chars for x in input_bytes])
    return nb_letters / len(input_bytes)

def is_probably_text(input_bytes):
    r = letter_ratio(input_bytes)
    return True if r>0.7 else False

In [35]:
is_probably_text(b"yo bro 12222222222?")

False

In [36]:
def attack_single_byte_xor(ciphertext):
    # a variable to keep track of the best candidate so far
    best = None
    for i in range(2**8): # for every possible key
        # converting the key from a number to a byte
        candidate_key = i.to_bytes(1, byteorder='big')
        keystream = candidate_key*len(ciphertext)
        candidate_message = bxor(ciphertext, keystream)
        nb_letters = sum([ x in ascii_text_chars for x in candidate_message])
        # if the obtained message has more letters than any other candidate before
        if best == None or nb_letters > best['nb_letters']:
            # store the current key and message as our best candidate so far
            best = {"message": candidate_message, 'nb_letters': nb_letters, 'key': candidate_key}
    return best

result = attack_single_byte_xor(ciphertext)

print('key:', result['key'])
print('message:', result['message'])
print('nb of letters:', result['nb_letters'])

(is_probably_text(result['message']))

key: b'X'
message: b"Cooking MC's like a pound of bacon"
nb of letters: 30


True

In [37]:
ciphertext

b'\x1b77316?x\x15\x1b\x7f+x413=x9x(7-6<x7>x:9;76'

In [38]:
def xor_with_character(input_string, char):
    char_byte = ord(char)  # Convert the character to its ASCII value
    result_bytes = bytes([byte ^ char_byte for byte in input_string.encode('utf-8')])  # XOR each byte with the character
    return result_bytes.hex()  # Convert the result to a hex string


In [39]:
sele = """
Milungi mishisha mibange selemani
Kujipostisha kama we boss yani
We selemani we sele sele sele  
"""
hex_cypher_sele = xor_with_character(sele, "Q")
hex_cypher_sele

'5b1c383d243f3638713c38223938223930713c3833303f36347122343d343c303f385b1a243b38213e222538223930713a303c3071263471333e22227128303f385b06347122343d343c303f387126347122343d347122343d347122343d3471715b'

In [40]:
cypher_sele = unhexlify(hex_cypher_sele)
cypher_sele

b'[\x1c8=$?68q<8"98"90q<830?64q"4=4<0?8[\x1a$;8!>"%8"90q:0<0q&4q3>""q(0?8[\x064q"4=4<0?8q&4q"4=4q"4=4q"4=4qq['

In [41]:
cracked_sele = attack_single_byte_xor(cypher_sele)

print('key🥷:', '\033[92m' + str(cracked_sele['key']) + '\033[0m')
message_bytes = cracked_sele['message']
message_str = message_bytes.decode('utf-8')
print('message🙅‍♂️:', '\033[94m' + message_str + '\033[0m')



key🥷: b'Q'
message🙅‍♂️: 
Milungi mishisha mibange selemani
Kujipostisha kama we boss yani
We selemani we sele sele sele  



**Challenge 4**
Detect single-character XOR
One of the 60-character strings in this file has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [42]:
class InvalidMessageException(Exception):
    pass

def attack_single_byte_xor(ciphertext):
    best = {"nb_letters": 0}
    for i in range(2**8):
        candidate_key = i.to_bytes(1, byteorder='big')
        candidate_message = bxor(ciphertext, candidate_key*len(ciphertext))
        nb_letters = sum([ x in ascii_text_chars for x in candidate_message])
        if nb_letters>best['nb_letters']:
            best = {"message": candidate_message, 'nb_letters': nb_letters, 'key': candidate_key}
    
    if best['nb_letters'] > 0.7*len(ciphertext):
        return best
    else:
        raise InvalidMessageException('best candidate message is: %s' % best['message'])

In [43]:
from os import urandom

try:
    # Instead of giving a real encrypted message we give random bytes
    attack_single_byte_xor(urandom(16))
except InvalidMessageException:
    print('Got an InvalidMessageException as expected')
else:
    print('No exception: something is wrong')

Got an InvalidMessageException as expected


In [44]:
with open('data/4.txt') as data_file:
    ciphertext_list = [
        # the 'strip' is to remove the "newline" character
        # which python keeps when reading a file line by line
        unhexlify(line.strip())
        for line in data_file
    ]

In [45]:
candidates = list()

for (line_nb, ciphertext) in enumerate(ciphertext_list):
    try:
        result = attack_single_byte_xor(ciphertext)

    except InvalidMessageException:
        pass
    else:
        candidates.append({
            'line_nb': line_nb,
            'ciphertext': ciphertext,
            'message': result['message'],
            'key': result["key"]
        })
        
if len(candidates) > 1:
    print("Error: more than one candidate")
    print(False)
else:
    for (key, value) in candidates[0].items():
        print(f'{key}: {value}')
    print(is_probably_text(candidates[0]['message']))

line_nb: 170
ciphertext: b'{ZB\x15A]TA\x15A]P\x15ETGAL\x15\\F\x15_@XE\\[R?'
message: b'Now that the party is jumping\n'
key: b'5'
True


In [48]:
x = 'Now that the party is jumping\n'
xor_with_character(x, "5")

'7b5a4215415d544115415d5015455447414c155c46155f4058455c5b523f'

In [47]:
hexlify(b'{ZB\x15A]TA\x15A]P\x15ETGAL\x15\\F\x15_@XE\\[R?')

b'7b5a4215415d544115415d5015455447414c155c46155f4058455c5b523f'